In [ ]:
import pandas as pd
import numpy as np
from glob2 import glob
import json

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stp
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v

from datetime import datetime
from datetime import date
from datetime import timedelta

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [1]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)

NameError: name 'pd' is not defined

In [8]:
import random


def cross_val_jsons(jsonFiles, isTest):
    # output = [json for json in jsonFiles]
    # print(random.choice(jsonFiles))
    if (isTest == True):
        test = random.choice(jsonFiles)
        jsonFiles.remove(test)
        train = [json for event in jsonFiles for json in event]
        return train, test
    else:
        data = [json for event in jsonFiles for json in event]
        return [data]

def extract_data(datas):
    data_lists = []
    isRumorLists = []
    for index, dataset in enumerate(datas):
        data_list = []
        isRumorList = []
        count = 0 # help var

        for jsonFile in dataset:
            count+=1
            if jsonFile.find("non-rumours") == -1:
                isRumorList.append(1)
            else:
                isRumorList.append(0)

            with open (jsonFile, 'r') as f:
                for l in f.readlines():
                    if not l.strip (): # skip empty lines
                        continue
                    json_data = json.loads(l)
                    # print (json_data,"\n\n")
                    data_list.append(json_data)

        isRumorLists.append(pd.DataFrame(isRumorList, columns=['isRumor']))
        data_lists.append(data_list)
    return data_lists, isRumorLists

def printRumor(route):
    if route.find("rumours") == -1:
        print('non-rumors')
    else:
        print('rumor')

In [9]:
def capitalratio(tweet_text):
    uppers = [l for l in tweet_text if l.isupper()]
    capitalratio = len(uppers) / len(tweet_text)
    return capitalratio 

def tweets2tokens(tweet_text):
    # Tokenizing
    urls = []
    tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', tweet_text.lower()))

    # Setting url value (whether the tweet contains http link) and filter http links
    url=0
    for token in tokens:
        if token.startswith('http'):
            url=1
    tokens = [token for token in tokens if not token.startswith('http')]

    ## Stemming
    # porter = PorterStemmer()
    # tokens = [porter.stem(token) for token in tokens]

    # Filtering Stop words
    # from nltk.corpus import stopwords
    # stop_words = set(stopwords.words('english'))
    # tokens = [token for token in tokens if not token in stop_words]

    return tokens, url

def getposcount(tokens):
    postag = []
    poscount = {}
    poscount['Noun']=0
    poscount['Verb']=0
    poscount['Adjective'] = 0
    poscount['Pronoun']=0
    poscount['FirstPersonPronoun']=0
    poscount['SecondPersonPronoun']=0
    poscount['ThirdPersonPronoun']=0
    poscount['Adverb']=0
    poscount['Numeral']=0
    poscount['Conjunction_inj']=0
    poscount['Particle']=0
    poscount['Determiner']=0
    poscount['Modal']=0
    poscount['Whs']=0
    Nouns = {'NN','NNS','NNP','NNPS'}
    Adverbs = {'RB','RBR','RBS'}
    Whs = {'WDT','WP','WRB'} # Composition of wh-determiner(that,what), wh-pronoun(who), wh-adverb(how)
    Verbs={'VB','VBP','VBZ','VBN','VBG','VBD','To'}
    first_person_pronouns=['i','I','me','my','mine','we','us','our','ours'] #'i',
    second_person_pronouns=['you','your','yours']
    third_person_pronouns=['he','she','it','him','her','it','his','hers','its','they','them','their','theirs']

    for word in tokens:
        w_lower=word.lower()
        if w_lower in first_person_pronouns:
            poscount['FirstPersonPronoun']+=1
        elif w_lower in second_person_pronouns:
            poscount['SecondPersonPronoun']+=1
        elif w_lower in third_person_pronouns:
            poscount['ThirdPersonPronoun']+=1
    
    postag = nltk.pos_tag(tokens)
    for g1 in postag:
        if g1[1] in Nouns:
            poscount['Noun'] += 1
        elif g1[1] in Verbs:
            poscount['Verb']+= 1
        elif g1[1]=='ADJ'or g1[1]=='JJ':
            poscount['Adjective']+=1
        elif g1[1]=='PRP' or g1[1]=='PRON' or g1[1]=='PRP$':
            poscount['Pronoun']+=1
        elif g1[1] in Adverbs or g1[1]=='ADV':
            poscount['Adverb']+=1
        elif g1[1]=='CD':
            poscount['Numeral']+=1
        elif g1[1]=='CC' or g1[1]=='IN':
            poscount['Conjunction_inj']+=1
        elif g1[1]=='RP':
            poscount['Particle']+=1
        elif g1[1]=='MD':
            poscount['Modal']+=1
        elif g1[1]=='DT':
            poscount['Determiner']+=1
        elif g1[1] in Whs:
            poscount['Whs']+=1
    return poscount

def contentlength(words):
    wordcount = len(words)
    return wordcount

def extract_urls(entities_dicts):
    if len(entities_dicts) < 1:
        return 0,0,0

    urls = []
    urls_expanded = []

    key = 'url'
    key2 = 'expanded_url'
    # print(len(entities_dict))
    for i in entities_dicts:
        urls.append(i[key])
        urls_expanded.append(i[key2])
    return 1, urls, urls_expanded

def flatten_tweets(tweets):
    """ Flattens out tweet dictionaries so relevant JSON is in a top-level dictionary. """
    tweets_list = []
    total_tokens_l = []

    # Iterate through each tweet
    for tweet_obj in tweets:
        output_f = dict()

        output_f['text']= tweet_obj['text']
        
        urls_dicts = tweet_obj['entities']['urls']
        # print(urls_dicts)

        output_f['hasURL'], output_f['urls'], output_f['urls_expanded'] = extract_urls(urls_dicts)
        
        # print(type(tweet_obj['user']))
        # print(tweet_obj['user'].contains_key('entities'))
        if ('url' in tweet_obj['user']):
            output_f['hasUserURL'] = 1
            output_f['user_url'] = tweet_obj['user']['url']
        elif ('entities' in tweet_obj['user']):
            # output_f['user_entity'] = tweet_obj['user']['entities']['url']['urls']
            # print(tweet_obj['user']['entities']['url']['urls'])
            # output_f['user_url'] = tweet_obj['user']['entities']['expanded_url']
            output_f['hasUserURL'] , _ , output_f['user_url'] = extract_urls(tweet_obj['user']['entities']['url']['urls'])
        else:
            # output_f['user_entity'] = None
            output_f['user_url'] = None
            output_f['hasUserURL'] = 0

        output_f['text_token'], output_f['isNotOnlyText'] = tweets2tokens(tweet_obj['text'])
        total_tokens_l.extend(output_f['text_token']) # append the tokens to list of total tokens

        '''POS Tagging'''
        pos_dict=getposcount(output_f['text_token'])
        output_f.update(pos_dict)

        output_f['char_count'] = len(output_f['text'])
        output_f['word_count'] = len(output_f['text_token'])

        output_f['has_question'] = "?" in output_f["text"]
        output_f['has_exclaim'] = "!" in output_f["text"]
        output_f['has_period'] = "." in output_f["text"]
    
        ''' User info'''
        # Store the user screen name in 'user-screen_name'
        # output_f['user-screen_name'] = tweet_obj['user']['screen_name']
        
        # Store the user location
        # output_f['user-location'] = tweet_obj['user']['location']

        acc_created = datetime.strptime(tweet_obj['user']['created_at'], '%a %b %d %H:%M:%S %z %Y')
        tweet_created = datetime.strptime(tweet_obj['created_at'], '%a %b %d %H:%M:%S %z %Y')
        age = (tweet_created - acc_created)
        # print(type(timedelta.total_seconds(age)))

        output_f['capital_ratio']=(capitalratio(tweet_obj['text']))

        # features=(capitalratio(data_list[0]['user']))
        output_f['tweet_count'] = np.log10(tweet_obj['user']['statuses_count'])
        output_f['listed_count'] = np.log10(tweet_obj['user']['listed_count'])
        output_f['follow_ratio'] = np.log10(tweet_obj['user']['followers_count'])
        output_f['age'] = int(timedelta.total_seconds(age)/86400)
        output_f['verified'] = tweet_obj['user']['verified']

        tweets_list.append(output_f)

    unk_tokens_l = list(set(total_tokens_l))
    print("Number of total tokens appeared: {}\nNumber of unique tokens appeared: {}\n".format(len(total_tokens_l), len(unk_tokens_l))) # number of tokens and unique tokens

    return tweets_list

In [15]:
charliehebdo_reaction = glob('../pheme-rnr-dataset/charliehebdo/**/reactions/*.json') 
ferguson_reaction = glob('../pheme-rnr-dataset/ferguson/**/reactions/*.json')
germanwing_scrash_reaction = glob('../pheme-rnr-dataset/germanwings-crash/**/reactions/*.json')
ottawashooting_reaction = glob('../pheme-rnr-dataset/ottawashooting/**/reactions/*.json')
sydneysiege_reaction = glob('../pheme-rnr-dataset/sydneysiege/**/reactions/*.json')

basic_reactionFiles = [charliehebdo_reaction, ferguson_reaction, germanwing_scrash_reaction, ottawashooting_reaction, sydneysiege_reaction]

In [11]:
charliehebdo_jsons = glob('../pheme-rumour-scheme-dataset/en/charliehebdo/**/source-tweets/*.json') 
ferguson_jsons = glob('../pheme-rumour-scheme-dataset/en/ferguson/**/source-tweets/*.json')
germanwing_scrash_jsons = glob('../pheme-rumour-scheme-dataset/en/germanwings-crash/**/source-tweets/*.json')
ottawashooting_jsons = glob('../pheme-rumour-scheme-dataset/en/ottawashooting/**/source-tweets/*.json')
sydneysiege_jsons = glob('../pheme-rumour-scheme-dataset/en/sydneysiege/**/source-tweets/*.json')

annotated_Files = [charliehebdo_reaction, ferguson_reaction, germanwing_scrash_reaction, ottawashooting_reaction, sydneysiege_reaction]

In [16]:
data_rt = cross_val_jsons(basic_reactionFiles, False)

In [ ]:
# jsonFiles = data_loader().getdata()
# data = cross_val_jsons(jsonFiles[0], isTest = False)
# data_lists, isRumorLists = extract_data(data)

data = data_lists[0]
data_y = isRumorLists[0]
print("(Data of Root tweets) Data: {}".format(len(data)))
print("(Data of Root tweets) Label: {}".format(len(data_y)))

In [20]:
data_rt_sample, isRumorLists = extract_data(data_rt)
data_rt_sample = data_rt_sample[0]
df_rt_y = isRumorLists[0]
print("(Data of Reaction tweets) Data: {}".format(len(data_rt_sample)))
print("(Data of Reaction tweets) Label: {}".format(len(df_rt_y)))

(Data of Reaction tweets) Data: 97410
(Data of Reaction tweets) Label: 97410


In [117]:
df_rt_X = pd.DataFrame(flatten_tweets(data_rt_sample))
df_rt_X[['has_question', 'has_exclaim', 'has_period','verified']] = df_rt_X[['has_question', 'has_exclaim', 'has_period','verified']].astype(int)

Number of total tokens appeared: 13515
Number of unique tokens appeared: 3179



In [118]:
df_rt_y.head()

,isRumor
0,1
1,1
2,1
3,1
4,1


In [119]:
for dataset in [df_rt_X]:
    dataset['listed_count'].replace([np.inf, -np.inf], np.nan, inplace=True)
    print(dataset['listed_count'].mean())
    print("Before fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))
    dataset['listed_count'].fillna(0,inplace=True)
    print("After fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))

1.03669578077241
Before fill: Does the dataset contain NaN value? True
After fill: Does the dataset contain NaN value? False


In [124]:
df_rt_X['text_token']

0      [haaretzcom, breaking, armed, man, takes, host...
1      [haaretzcom, ahmethez, to, kill, is, right, ho...
2      [haaretzcom, ahmethez, play, back, infront, of...
3      [ohohyesyesnono, haaretzcom, bots, will, conqu...
4      [haaretzcom, ahmethez, you, must, be, paranoid...
                             ...                        
995    [rt, afp, breaking, at, least, two, killed, in...
996    [oh, this, is, dreadful, afp, breaking, at, le...
997    [afp, breaking, at, least, two, killed, in, ho...
998    [rtafp, breaking, at, least, two, killed, in, ...
999    [afp, kevinvanmoortel, genoegggggg, stop, erme...
Name: text_token, Length: 1000, dtype: object

In [154]:
list_low = []
list_num = []

for index, tokens in enumerate(df_rt_X['text_token']):
    if len(tokens) < 3:
        list_low.append(tokens)
        list_num.append(index)
        # print(index, tokens)
        # print(len(tokens))
    # print(index, tokens)

In [155]:
df_rt_X.iloc[27]

text                   @bouckap @porliniers mucho Liniers http://t.co...
text_token                         [bouckap, porliniers, mucho, liniers]
Noun                                                                   3
Verb                                                                   1
Adjective                                                              0
Pronoun                                                                0
FirstPersonPronoun                                                     0
SecondPersonPronoun                                                    0
ThirdPersonPronoun                                                     0
Adverb                                                                 0
Numeral                                                                0
Conjunction_inj                                                        0
Particle                                                               0
Determiner                                         

In [151]:
print("Number of text containing less than 4 tokens:",df_rt_X.iloc[list_num]['text'].count())
df_rt_X.iloc[list_num]

Number of text containing less than 4 tokens: 38


,text,text_token,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,Numeral,Conjunction_inj,Particle,Determiner,Modal,Whs,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,follow_ratio,age,verified
47,@SkyNews @ProSyndicate,"[skynews, prosyndicate]",2,0,0,0,0,0,0,0,0,0,0,0,0,0,22,2,0,0,0,0.181818,2.978637,0.602060,2.445604,196,0
48,@SkyNews https://t.co/qumtYObGgK,[skynews],1,0,0,0,0,0,0,0,0,0,0,0,0,0,32,1,0,0,1,0.187500,2.271842,0.000000,1.342423,222,0
81,@FoxNews @ABOUBAKRLAWFIRM,"[foxnews, aboubakrlawfirm]",2,0,0,0,0,0,0,0,0,0,0,0,0,0,25,2,0,0,0,0.680000,4.121034,1.000000,2.489958,1062,0
123,@FRANCE24,[france24],1,0,0,0,0,0,0,0,0,0,0,0,0,0,9,1,0,0,0,0.666667,3.189771,0.903090,2.348305,2064,0
129,@FRANCE24,[france24],1,0,0,0,0,0,0,0,0,0,0,0,0,0,9,1,0,0,0,0.666667,1.785330,0.000000,1.041393,48,0
130,@FRANCE24,[france24],1,0,0,0,0,0,0,0,0,0,0,0,0,0,9,1,0,0,0,0.666667,3.658584,0.000000,2.029384,586,0
139,@SkyNews Savages,"[skynews, savages]",2,0,0,0,0,0,0,0,0,0,0,0,0,0,16,2,0,0,0,0.187500,2.271842,0.000000,1.397940,88,0
150,@RT_com @paolavela https://t.co/Beehnoo6p5,"[rtcom, paolavela]",2,0,0,0,0,0,0,0,0,0,0,0,0,0,42,2,0,0,1,0.071429,4.523304,1.278754,3.019947,2532,0
155,@RT_com dasyatnya,"[rtcom, dasyatnya]",2,0,0,0,0,0,0,0,0,0,0,0,0,0,17,2,0,0,0,0.117647,1.544068,0.000000,0.954243,333,0
276,@geetaSkapoor yes,"[geetaskapoor, yes]",1,0,1,0,0,0,0,0,0,0,0,0,0,0,17,2,0,0,0,0.058824,4.769348,1.301030,2.913284,1001,0


In [136]:
pd.DataFrame(list_low)

,0,1
0,skynews,prosyndicate
1,skynews,None
2,foxnews,aboubakrlawfirm
3,france24,None
4,france24,None
...,...,...
33,ironyisfunny8,None
34,bbcbreaking,again
35,bbcbreaking,bbcworld
36,bbcbreaking,None


In [138]:
vectorize(pd.DataFrame(list_low))

array([], dtype=float64)

In [98]:
word2vec_ = w2v.Word2Vec.load('w2v_model')
word_vectors = word2vec_.wv
# vocabs = word_vectors.vocab.keys()

In [99]:
w2v_vectors = word_vectors.vectors # here you load vectors for each word in your model
w2v_indices = {word: word_vectors.vocab[word].index for word in word_vectors.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [100]:
import copy
df_rt_X['text_token_vec'] = copy.deepcopy(df_rt_X['text_token'])

for index, sentence in enumerate(df_rt_X['text_token_vec']):
    df_rt_X['text_token_vec'][index] = vectorize(sentence).mean(axis=0)
    # df['text_token_vec'][0].mean(axis=0)

# new =  [i for i in df_rt_X['text_token_vec']]


# df_rt_X[['text_token','text_token_vec']].head()
# df_test_X['text_token_vec']

# df_test_avg = pd.DataFrame(new)
# df_test_avg = pd.DataFrame(np.ndarray(new).values)

new = df_rt_X['text_token_vec']

# for i, row in enumerate(new):
#     df_rt_Xpd.DataFrame(new.values.tolist()[0]).add_prefix('token_avg')# df_test_avg = pd.DataFrame(df_rt_X.values.tolist()).add_prefix('token_avg')
# # df_test_avg = pd.DataFrame(pd.Series(df_rt_X['text_token_vec']).values.tolist()).add_prefix('token_avg')

# df_rt_X = df_rt_X.join(df_test_avg).drop('text_token_vec',axis=1)
# df_rt_X.drop('text_token_vec',axis=1, inplace=True)

In [110]:
df_rt_X

,text,text_token,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,Numeral,Conjunction_inj,Particle,Determiner,Modal,Whs,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,follow_ratio,age,verified,text_token_vec
0,“@haaretzcom: BREAKING: Armed man takes hostag...,"[haaretzcom, breaking, armed, man, takes, host...",9,2,0,0,0,0,0,0,0,2,0,0,0,0,116,13,0,0,1,0.120690,5.010406,1.518514,3.112605,1391,0,"[-0.33929467, 0.03344092, 0.05817368, 0.065974..."
1,@haaretzcom @AhmetHez to kill is right? How da...,"[haaretzcom, ahmethez, to, kill, is, right, ho...",7,5,2,4,0,4,0,1,0,2,0,0,0,3,133,25,1,0,0,0.022556,3.361161,0.000000,2.212188,2,0,"[-0.203123, 0.13672417, 0.036033094, 0.0193106..."
2,@haaretzcom @AhmetHez play back infront of ur ...,"[haaretzcom, ahmethez, play, back, infront, of...",10,2,2,4,0,2,2,1,0,5,0,0,1,2,140,27,0,0,0,0.014286,3.361161,0.000000,2.212188,2,0,"[-0.20826234, 0.13303766, 0.09189266, 0.027073..."
3,@ohohyesyesnono @haaretzcom Bots will conquest...,"[ohohyesyesnono, haaretzcom, bots, will, conqu...",4,1,0,0,0,0,0,0,0,0,0,1,1,0,61,8,0,0,1,0.016393,5.056325,0.954243,2.530200,68,0,"[-0.16941941, 0.15662226, 0.038318496, 0.15657..."
4,@haaretzcom @AhmetHez you must be paranoid to ...,"[haaretzcom, ahmethez, you, must, be, paranoid...",5,6,3,3,0,2,1,0,0,1,1,0,1,0,130,22,0,0,0,0.100000,3.361161,0.000000,2.212188,2,0,"[-0.2128465, 0.14920598, 0.06260962, 0.0504627..."
5,“@haaretzcom: BREAKING: Armed man takes hostag...,"[haaretzcom, breaking, armed, man, takes, host...",8,2,0,0,0,0,0,0,0,2,0,0,0,0,103,12,0,0,1,0.155340,4.639845,1.612784,4.064346,2003,0,"[-0.33929467, 0.03344092, 0.05817368, 0.065974..."
6,@haaretzcom @AhmetHez cell phones are useful u...,"[haaretzcom, ahmethez, cell, phones, are, usef...",6,4,1,2,0,2,0,2,0,0,0,0,0,1,99,18,0,0,0,0.020202,3.361161,0.000000,2.212188,2,0,"[-0.2405014, 0.17109896, 0.08023744, 0.0673134..."
7,Religion; can it get any more dumb arsed?\n\n“...,"[religion, can, it, get, any, more, dumb, arse...",10,4,0,1,0,0,1,0,0,2,0,1,1,0,125,20,1,0,1,0.088000,4.362105,1.491362,2.656098,1699,0,"[-0.30599895, 0.07778995, 0.06438117, 0.101598..."
8,@haaretzcom @AhmetHez crazy parents think they...,"[haaretzcom, ahmethez, crazy, parents, think, ...",7,4,1,2,0,0,2,1,0,2,0,0,0,0,119,18,0,0,0,0.016807,3.361161,0.000000,2.212188,2,0,"[-0.20803127, 0.13690697, 0.11683881, 0.070493..."
9,@haaretzcom @AhmetHez when Soviet Union collap...,"[haaretzcom, ahmethez, when, soviet, union, co...",5,5,2,1,0,0,1,1,0,2,0,2,0,1,131,20,0,0,0,0.045802,3.361161,0.000000,2.212188,2,0,"[-0.18680982, 0.10569486, 0.09245623, 0.053127..."


In [90]:
np.concatenate( new, axis=0 )

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 27 has 0 dimension(s)

In [78]:
new

0      [-0.33929467, 0.03344092, 0.05817368, 0.065974...
1      [-0.203123, 0.13672417, 0.036033094, 0.0193106...
2      [-0.20826234, 0.13303766, 0.09189266, 0.027073...
3      [-0.16941941, 0.15662226, 0.038318496, 0.15657...
4      [-0.2128465, 0.14920598, 0.06260962, 0.0504627...
                             ...                        
995    [-0.28859487, -0.016051898, 0.059103392, 0.056...
996    [-0.26605442, -0.00085825985, 0.0729878, 0.062...
997    [-0.30163282, -0.031242117, 0.047385555, 0.060...
998    [-0.27851185, -0.030041477, 0.043500897, 0.051...
999    [-0.2202898, 0.04332359, 0.07064113, 0.0510248...
Name: text_token_vec, Length: 1000, dtype: object

In [76]:
df_test_avg

,token_avg0,token_avg1,token_avg2,token_avg3,token_avg4,token_avg5,token_avg6,token_avg7,token_avg8,token_avg9,token_avg10,token_avg11,token_avg12,token_avg13,token_avg14,token_avg15,token_avg16,token_avg17,token_avg18,token_avg19,token_avg20,token_avg21,token_avg22,token_avg23,token_avg24,token_avg25,token_avg26,token_avg27
0,“@haaretzcom: BREAKING: Armed man takes hostag...,"[haaretzcom, breaking, armed, man, takes, host...",9,2,0,0,0,0,0,0,0,2,0,0,0,0,116,13,0,0,1,0.120690,5.010406,1.518514,3.112605,1391,0,"[-0.33929467, 0.03344092, 0.05817368, 0.065974..."
1,@haaretzcom @AhmetHez to kill is right? How da...,"[haaretzcom, ahmethez, to, kill, is, right, ho...",7,5,2,4,0,4,0,1,0,2,0,0,0,3,133,25,1,0,0,0.022556,3.361161,0.000000,2.212188,2,0,"[-0.203123, 0.13672417, 0.036033094, 0.0193106..."
2,@haaretzcom @AhmetHez play back infront of ur ...,"[haaretzcom, ahmethez, play, back, infront, of...",10,2,2,4,0,2,2,1,0,5,0,0,1,2,140,27,0,0,0,0.014286,3.361161,0.000000,2.212188,2,0,"[-0.20826234, 0.13303766, 0.09189266, 0.027073..."
3,@ohohyesyesnono @haaretzcom Bots will conquest...,"[ohohyesyesnono, haaretzcom, bots, will, conqu...",4,1,0,0,0,0,0,0,0,0,0,1,1,0,61,8,0,0,1,0.016393,5.056325,0.954243,2.530200,68,0,"[-0.16941941, 0.15662226, 0.038318496, 0.15657..."
4,@haaretzcom @AhmetHez you must be paranoid to ...,"[haaretzcom, ahmethez, you, must, be, paranoid...",5,6,3,3,0,2,1,0,0,1,1,0,1,0,130,22,0,0,0,0.100000,3.361161,0.000000,2.212188,2,0,"[-0.2128465, 0.14920598, 0.06260962, 0.0504627..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,RT @AFP: #BREAKING At least two killed in host...,"[rt, afp, breaking, at, least, two, killed, in...",7,2,0,0,0,0,0,0,1,3,0,0,0,0,77,14,0,0,0,0.194805,3.722881,2.033424,3.282849,968,0,"[-0.28859487, -0.016051898, 0.059103392, 0.056..."
996,Oh this is dreadful... @AFP #BREAKING At least...,"[oh, this, is, dreadful, afp, breaking, at, le...",5,3,1,0,0,0,0,1,1,4,0,1,0,0,96,17,0,0,1,0.145833,4.828847,2.833784,3.820924,1728,0,"[-0.26605442, -0.00085825985, 0.0729878, 0.062..."
997,“@AFP: #BREAKING At least two killed in hostag...,"[afp, breaking, at, least, two, killed, in, ho...",7,2,0,0,0,0,0,0,1,3,0,0,0,0,87,14,0,0,0,0.160920,3.883093,1.913814,3.282396,1418,0,"[-0.30163282, -0.031242117, 0.047385555, 0.060..."
998,RT“@AFP: #BREAKING At least two killed in host...,"[rtafp, breaking, at, least, two, killed, in, ...",6,2,0,0,0,0,0,0,1,3,0,0,0,0,78,13,0,0,0,0.192308,4.253895,1.322219,2.790988,1589,0,"[-0.27851185, -0.030041477, 0.043500897, 0.051..."


In [56]:
new

.55976038e-02, -4.89092171e-02, -5.63075580e-02,
        -9.50548351e-02,  7.52099007e-02, -1.75710872e-01, -1.43593326e-01,
         2.68511884e-02, -1.51181920e-02, -6.62887096e-02,  6.37823790e-02,
        -1.09970093e-01,  2.20689364e-02,  8.16102475e-02, -4.58662771e-02,
        -1.29427746e-01,  1.04344249e-01, -1.62940115e-01, -1.44394949e-01,
         1.42450094e-01, -1.56781613e-03,  1.48575783e-01,  1.29669830e-01,
        -1.14570335e-01, -1.10423662e-01, -8.29554275e-02,  3.79282981e-02,
        -8.40811953e-02,  3.09146792e-02, -1.69605121e-01, -3.61536853e-02,
         1.26682341e-01,  5.48496470e-02, -1.11219548e-01, -1.87759958e-02,
         1.00406200e-01, -2.65115174e-04,  7.41460100e-02, -2.79345810e-02,
        -3.76502681e-03, -2.23233216e-02, -4.53662798e-02, -1.70536358e-02,
         5.44243380e-02, -1.14275202e-01, -4.73478511e-02,  6.86433315e-02,
         1.87801629e-01,  3.03938277e-02, -2.27393042e-02, -2.08856841e-03,
         1.60599072e-02, -3.96504700e-0

In [22]:
df_test_avg

token_avg0      [-0.33929467, 0.03344092, 0.05817368, 0.065974...
token_avg1      [-0.203123, 0.13672417, 0.036033094, 0.0193106...
token_avg2      [-0.20826234, 0.13303766, 0.09189266, 0.027073...
token_avg3      [-0.16941941, 0.15662226, 0.038318496, 0.15657...
token_avg4      [-0.2128465, 0.14920598, 0.06260962, 0.0504627...
                                      ...                        
token_avg995    [-0.28859487, -0.016051898, 0.059103392, 0.056...
token_avg996    [-0.26605442, -0.00085825985, 0.0729878, 0.062...
token_avg997    [-0.30163282, -0.031242117, 0.047385555, 0.060...
token_avg998    [-0.27851185, -0.030041477, 0.043500897, 0.051...
token_avg999    [-0.2202898, 0.04332359, 0.07064113, 0.0510248...
Length: 1000, dtype: object

---

In [16]:
df_rt_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 1000 non-null   object 
 1   text_token           1000 non-null   object 
 2   Noun                 1000 non-null   int64  
 3   Verb                 1000 non-null   int64  
 4   Adjective            1000 non-null   int64  
 5   Pronoun              1000 non-null   int64  
 6   FirstPersonPronoun   1000 non-null   int64  
 7   SecondPersonPronoun  1000 non-null   int64  
 8   ThirdPersonPronoun   1000 non-null   int64  
 9   Adverb               1000 non-null   int64  
 10  Numeral              1000 non-null   int64  
 11  Conjunction_inj      1000 non-null   int64  
 12  Particle             1000 non-null   int64  
 13  Determiner           1000 non-null   int64  
 14  Modal                1000 non-null   int64  
 15  Whs                  1000 non-null   in

In [13]:
df_rt_X.head()

,text,text_token,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,Numeral,Conjunction_inj,Particle,Determiner,Modal,Whs,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,follow_ratio,age,verified
0,“@haaretzcom: BREAKING: Armed man takes hostag...,"[haaretzcom, breaking, armed, man, takes, host...",9,2,0,0,0,0,0,0,0,2,0,0,0,0,116,13,0,0,1,0.120690,5.010406,1.518514,3.112605,1391,0
1,@haaretzcom @AhmetHez to kill is right? How da...,"[haaretzcom, ahmethez, to, kill, is, right, ho...",7,5,2,4,0,4,0,1,0,2,0,0,0,3,133,25,1,0,0,0.022556,3.361161,0.000000,2.212188,2,0
2,@haaretzcom @AhmetHez play back infront of ur ...,"[haaretzcom, ahmethez, play, back, infront, of...",10,2,2,4,0,2,2,1,0,5,0,0,1,2,140,27,0,0,0,0.014286,3.361161,0.000000,2.212188,2,0
3,@ohohyesyesnono @haaretzcom Bots will conquest...,"[ohohyesyesnono, haaretzcom, bots, will, conqu...",4,1,0,0,0,0,0,0,0,0,0,1,1,0,61,8,0,0,1,0.016393,5.056325,0.954243,2.530200,68,0
4,@haaretzcom @AhmetHez you must be paranoid to ...,"[haaretzcom, ahmethez, you, must, be, paranoid...",5,6,3,3,0,2,1,0,0,1,1,0,1,0,130,22,0,0,0,0.100000,3.361161,0.000000,2.212188,2,0


In [17]:
df_rt_y.head()

,isRumor
0,1
1,1
2,1
3,1
4,1
